In [ ]:
import os
import json
from datetime import datetime
import pandas as pd
import math

In [ ]:
# STEP1: sort nodes by time

time_name = 'time'
# time_name = 'published_at'
def sort_nodes(nodes):
    available_nodes = []
    not_available_nodes = []

    for node in nodes:
        if node[time_name] != "Not available":
            available_nodes.append(node)
        else:
            not_available_nodes.append(node)

    available_nodes.sort(key=lambda x: datetime.fromisoformat(x[time_name].replace("Z", "+00:00")))
    not_available_nodes.sort(key=lambda x: x['name'].lower())
    sorted_nodes = available_nodes + not_available_nodes

    return sorted_nodes

def process_json_files_in_folder(folder_path):
    json_files = [f for f in os.listdir(folder_path) if f.endswith('.json')]
    
    for json_file in json_files:
        with open(os.path.join(folder_path, json_file), 'r', encoding='utf-8') as f:
            data = json.load(f)

        if 'nodes' in data:
            sorted_nodes = sort_nodes(data['nodes'])
            data['nodes'] = sorted_nodes

        with open(os.path.join(folder_path, json_file), 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=4)

        print(f"Processed and sorted nodes in {json_file}")

def process_multiple_folders(folder_paths):
    for folder_path in folder_paths:
        if os.path.exists(folder_path):
            process_json_files_in_folder(folder_path)
        else:
            print(f"Folder {folder_path} does not exist.")

folder_paths = [
# # # dependent-hidden graph
#   'KG/Dependency_Edge/metadata_dep/npm',
#   'KG/Dependency_Edge/sourcecode_dep/npm',
#   'KG/Dependency_Edge/metadata_dep/pypi',
#   'KG/Dependency_Edge/sourcecode_dep/pypi',
#   'KG/Dependency_Edge/metadata_dep/ruby',
#   'KG/Dependency_Edge/sourcecode_dep/ruby'

# co-existing graph
  'KG/Co-existing_Edge/source_report/npm',
  'KG/Co-existing_Edge/WebCrawler_report/npm',
  'KG/Co-existing_Edge/source_report/pypi',
  'KG/Co-existing_Edge/WebCrawler_report/pypi',
  'KG/Co-existing_Edge/source_report/ruby',
  'KG/Co-existing_Edge/WebCrawler_report/ruby'
]

process_multiple_folders(folder_paths)


In [ ]:
# STEP2: generate evolution for depdent-hidden/co-existing graphs
def replace_nan_with_none(data):
    if isinstance(data, dict):
        return {key: replace_nan_with_none(value) for key, value in data.items()}
    elif isinstance(data, list):
        return [replace_nan_with_none(item) for item in data]
    elif data is None or (isinstance(data, float) and math.isnan(data)):
        return None
    else:
        return data
    
total_name_change = 0
total_version_change = 0
total_description_change = 0
total_dependencies_change = 0
total_code_change = 0
total_change_num = 0

def compare_nodes(prev_node, curr_node):
    global total_name_change, total_version_change, total_description_change, total_dependencies_change, total_code_change, total_change_num

    prev_node = replace_nan_with_none(prev_node)
    curr_node = replace_nan_with_none(curr_node)

    if prev_node['name'] != curr_node['name']:
        total_name_change += 1
    if (prev_node['name'] == curr_node['name']) and (prev_node.get('version', 'none') != curr_node.get('version', 'none')):
        total_version_change += 1
    if (prev_node.get('SHA-256', 'none') != 'none') and (curr_node.get('SHA-256', 'none') != 'none') and (prev_node['description'] != curr_node['description']):
        total_description_change += 1
    if (prev_node.get('SHA-256', 'none') != 'none') and (curr_node.get('SHA-256', 'none') != 'none') and (prev_node.get('SHA-256', 'none') != curr_node.get('SHA-256', 'none')):
        total_code_change += 1
    if (prev_node.get('SHA-256', 'none') != 'none') and (curr_node.get('SHA-256', 'none') != 'none') and (prev_node['dependencies'] != curr_node['dependencies']):
        total_dependencies_change += 1

def process_json_files_in_folder(folder_path):
    global total_name_change, total_version_change, total_description_change, total_dependencies_change, total_code_change
    
    json_files = [f for f in os.listdir(folder_path) if f.endswith('.json')]
    
    for json_file in json_files:
        with open(os.path.join(folder_path, json_file), 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        nodes = data.get('nodes', [])
        
        for i in range(1, len(nodes)):
            prev_node = nodes[i - 1]
            curr_node = nodes[i]
            compare_nodes(prev_node, curr_node)

def process_multiple_folders(folder_paths):
    for folder_path in folder_paths:
        process_json_files_in_folder(folder_path)
    
    total_change_num = total_name_change+total_version_change
    final_results = {
        # "CN": total_name_change / (total_name_change + total_version_change + total_description_change + total_dependencies_change + total_code_change),
        # "CV": total_version_change / (total_name_change + total_version_change + total_description_change + total_dependencies_change + total_code_change),
        # "CD": total_description_change / (total_name_change + total_version_change + total_description_change + total_dependencies_change + total_code_change),
        # "CDep": total_dependencies_change / (total_name_change + total_version_change + total_description_change + total_dependencies_change + total_code_change),
        # "CC": total_code_change / (total_name_change + total_version_change + total_description_change + total_dependencies_change + total_code_change)
        "CN": total_name_change / total_change_num,
        "CV": total_version_change / total_change_num,
        "CD": total_description_change / total_change_num,
        "CDep": total_dependencies_change / total_change_num,
        "CC": total_code_change / total_change_num
    }

    # Convert to DataFrame
    df = pd.DataFrame(list(final_results.items()), columns=['Change Type', 'Value'])
    
    output_file = './evolution.xlsx'
    df.to_excel(output_file, index=False)
    
    print(f"Final aggregated results saved to {output_file}")
    print("Final aggregated results across all folders:", final_results)

folder_paths = [
# # # dependent-hidden graph
#   'KG/Dependency_Edge/metadata_dep/npm',
#   'KG/Dependency_Edge/sourcecode_dep/npm',
#   'KG/Dependency_Edge/metadata_dep/pypi',
#   'KG/Dependency_Edge/sourcecode_dep/pypi',
#   'KG/Dependency_Edge/metadata_dep/ruby',
#   'KG/Dependency_Edge/sourcecode_dep/ruby'

# co-existing graph
  'KG/Co-existing_Edge/source_report/npm',
  'KG/Co-existing_Edge/WebCrawler_report/npm',
  'KG/Co-existing_Edge/source_report/pypi',
  'KG/Co-existing_Edge/WebCrawler_report/pypi',
  'KG/Co-existing_Edge/source_report/ruby',
  'KG/Co-existing_Edge/WebCrawler_report/ruby'
]

process_multiple_folders(folder_paths)


In [ ]:
# STEP2: generate evolution for similar  graphs
def replace_nan_with_none(node):
    return {k: (None if v != v else v) for k, v in node.items()}  

def process_multiple_excel(excel_files, output_file):
    all_results = []

    for excel_file in excel_files:
        df = pd.read_excel(excel_file)

        required_columns = ['cluster', 'published_at', 'name', 'version', 'SHA-256', 'description', 'dependencies']
        if not all(col in df.columns for col in required_columns):
            print(f"Error: The Excel file {excel_file} must contain 'cluster', 'published_at', 'name', 'version', 'SHA-256', 'description', and 'dependencies' columns.")
            continue
        
        total_name_change = 0
        total_version_change = 0
        total_description_change = 0
        total_dependencies_change = 0
        total_code_change = 0
        total_change_num = 0

        for cluster, group in df.groupby('cluster'):
            group_sorted = group.sort_values(by='published_at')

            prev_node = None
            for index, curr_node in group_sorted.iterrows():
                if prev_node is not None:
                    prev_node = replace_nan_with_none(prev_node)
                    curr_node = replace_nan_with_none(curr_node)

                    if prev_node['name'] != curr_node['name']:
                        total_name_change += 1
                    if prev_node['name'] == curr_node['name'] and prev_node.get('version', 'none') != curr_node.get('version', 'none'):
                        total_version_change += 1
                    if prev_node.get('SHA-256', 'none') != 'none' and curr_node.get('SHA-256', 'none') != 'none' and prev_node['description'] != curr_node['description']:
                        total_description_change += 1
                    if prev_node.get('SHA-256', 'none') != 'none' and curr_node.get('SHA-256', 'none') != 'none' and prev_node.get('SHA-256', 'none') != curr_node.get('SHA-256', 'none'):
                        total_code_change += 1
                    if prev_node.get('SHA-256', 'none') != 'none' and curr_node.get('SHA-256', 'none') != 'none' and prev_node['dependencies'] != curr_node['dependencies']:
                        total_dependencies_change += 1

                prev_node = curr_node

        total_change_num = total_name_change + total_version_change 
        all_results.append([excel_file,total_name_change, total_version_change, total_description_change, total_dependencies_change, total_code_change, total_change_num])

    result_df = pd.DataFrame(all_results, columns=['File_name','Total Name Change', 'Total Version Change', 'Total Description Change', 'Total Dependencies Change', 'Total Code Change', 'Total Change Num'])

    result_df.to_excel(output_file, index=False)
    print(f"Updated Excel file has been saved to {output_file}")

excel_files = [
    'KG/Similar_Edge/npm_end.xlsx', 
    'KG/Similar_Edge/pypi_end.xlsx',  
    'KG/Similar_Edge/ruby_end.xlsx',
]

output_excel_path = './evolution.xlsx'  
process_multiple_excel(excel_files, output_excel_path)
